In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

#### Obtener el DF principal: Movies

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_foler_path}/movies").filter("year_release_date >= 2000") \
                                .filter(f"file_date = '{v_file_date}'")

#### Obtener los datos de Languages

In [0]:
movie_languages_df = spark.read.format("delta").load(f"{silver_foler_path}/movie_languages") \
                                        .filter(f"file_date = '{v_file_date}'")
languages_df = spark.read.format("delta").load(f"{silver_foler_path}/language")

#### Obtener los datos de Genre

In [0]:
genre_df = spark.read.format("delta").load(f"{silver_foler_path}/genre")
movie_genre = spark.read.format("delta").load(f"{silver_foler_path}/movie_genre") \
                    .filter(f"file_date = '{v_file_date}'")

#### Obtener Tablas Cruzadas

In [0]:
languages_final = movie_languages_df.join(languages_df,movie_languages_df.language_id == languages_df.language_id,"inner") \
                                        .select(movie_languages_df.movie_id, movie_languages_df.language_id, languages_df.language_name)

In [0]:
genre_final = movie_genre.join(genre_df,movie_genre.genre_id == genre_df.genre_id,"inner") \
                                .select(movie_genre.movie_id, movie_genre.genre_id, genre_df.genre_name)

#### Obtener tabla final

In [0]:
  movie_languages_final = movie_df.join(languages_final,movie_df.movie_id == languages_final.movie_id,"inner") \
                          .select(movie_df.title, movie_df.duration_time, movie_df.release_date,movie_df.vote_average,
                                  languages_final.language_name,movie_df.movie_id, languages_final.language_id)

In [0]:
from pyspark.sql.functions import current_timestamp, lit

movie_final = movie_languages_final.join(genre_final, movie_languages_final.movie_id == genre_final.movie_id,"inner") \
                                  .select(movie_languages_final.movie_id,"genre_id","language_id",movie_languages_final.title, movie_languages_final.duration_time, movie_languages_final.release_date,
                                          movie_languages_final.vote_average, movie_languages_final.language_name, genre_final.genre_name).withColumn("created_date",lit(v_file_date)).orderBy(movie_languages_final.release_date, ascending=False)

In [0]:
display(movie_final)

In [0]:
#overwrite_partition(movie_final, "movie_gold", "results_movie_genre_language", "created_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.genre_id = src.genre_id AND tgt.created_date = src.created_date'
merge_delta_lake(movie_final, "movie_gold", "results_movie_genre_language", gold_folder_path, merge_condition, "created_date")


In [0]:
%sql
SELECT * FROM movie_gold.results_movie_genre_language